In [1]:
import pandas as pd
import os
import re
import numpy as np

import pickle

import sys
sys.path.insert(0, '/media/andrii/earth/Katia/CS_MasterThesis/Red_links_Project_for_Wiki_draft/py')
import functions

import time

In [2]:
from multiprocessing import Pool

In [3]:
PATH_TO_DATA_UK = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki_20180920/'
PATH_TO_DATA_EN = '/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/enwiki_20180920/'
PATH_TO_DATA_RL = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/'
PATH_TO_WIKIDATA_UK = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_wikidata/'
PATH_TO_WIKIDATA_EN = '/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/en_wikidata/'

### Get Wikidata-Wikipedia correspondences for English pages

In [4]:
enwikidata = pd.read_csv(PATH_TO_WIKIDATA_EN+'enwikidataItem.csv', index_col=0)

/home/andrii/apps/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
en_id_name = pd.read_csv(PATH_TO_DATA_EN+'enwiki-20180920-id_name.csv', sep='^')

In [6]:
en_wikidata_wikipedia = pd.merge(enwikidata, en_id_name, how='left', left_on=['ips_site_page'], right_on=['title'])

In [7]:
en_wikidata_wikipedia = en_wikidata_wikipedia[['ips_item_id', 'title', 'id']]
en_wikidata_wikipedia.columns = ['wikidata_id', 'title', 'en_wikipedia_id']

In [8]:
en_wikidata_wikipedia.head()

,wikidata_id,title,en_wikipedia_id
0,3466056,!PAUS3,33744797.0
1,4540212,"""And"" theory of conservatism",9003666.0
2,3284078,"""Buzz!!"" The Movie",13623996.0
3,4540290,"""In"" Jazz for the Culture Set",32066421.0
4,4540291,"""Isis"" of the Suebi",31500663.0


In [9]:
en_wikidata_wikipedia.shape

(8005914, 3)

In [10]:
en_wikidata_wikipedia = en_wikidata_wikipedia.drop_duplicates()

In [11]:
en_wikidata_wikipedia.shape

(7928332, 3)

### Load candidate pairs

In [12]:
files = [f for f in os.listdir(PATH_TO_DATA_RL+'candidates/') if re.match(r'[a-zA-Z]+_\d+.csv', f)]

In [13]:
candidate_pairs = functions.load_csvs_to_df_general(PATH_TO_DATA_RL+'candidates/', files)

101
1/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_15.csv
2/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_93.csv
3/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_68.csv
4/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_45.csv
5/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_23.csv
6/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_79.csv
7/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_74.csv
8/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_42.csv
9/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_88.csv
10/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/cand

81/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_49.csv
82/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_84.csv
83/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_51.csv
84/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_43.csv
85/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_8.csv
86/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_61.csv
87/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_39.csv
88/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_64.csv
89/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/candidates/candidates_41.csv
90/101
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/

In [14]:
candidate_pairs.shape

(2964382, 2)

### Load en pagelinks and prepare data for counting concurrent links of candidate pages

In [15]:
# load en links for all pages
en_pagelinks_np = np.load(PATH_TO_DATA_EN+'results_save.npy')

en_pagelinks = pd.DataFrame(en_pagelinks_np)
en_pagelinks.columns = ['parent_id', 'child_id']

In [20]:
# RUN ONCE! converts en pagelinks df to dictionary and saves it


# en_pagelinks_dict = {k: v['child_id'].tolist() for k,v in en_pagelinks.groupby('parent_id')}

# with open(PATH_TO_DATA_EN+'en_pagelinks_dict.p', 'wb') as fp:
#     pickle.dump(en_pagelinks_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
# load pickle file
with open(PATH_TO_DATA_EN+'en_pagelinks_dict.p', 'rb') as fp:
    en_pagelinks_dict = pickle.load(fp)

In [16]:
# create {title: id} dict for candidates
candidates_list = candidate_pairs['candidate'].drop_duplicates().values.tolist()

print('number of unique candidates is', len(candidates_list))

en_candidates_df = en_wikidata_wikipedia[en_wikidata_wikipedia['title'].isin(candidates_list)]

en_candidates_df.head()

candidates_dict = dict(zip(en_candidates_df['title'], en_candidates_df['en_wikipedia_id']))


pool = Pool()
candidates_id_list = pool.map(candidates_dict.get, candidates_list)
pool.close()
pool.join()

number of unique candidates is 130152


In [19]:
with open(PATH_TO_DATA_EN+'candidates_title_id_dict.p', 'wb') as fp:
    pickle.dump(candidates_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
# RUN ONLY ONCE!! Creates a dict with filtered en pagelinks and saves to a file


# filter pagelinks keeping only with parent pages of candidates 
pagelinks_only_where_candidates = pd.merge(en_pagelinks, en_candidates_df, how='left', left_on=['child_id'], right_on=['en_wikipedia_id'])

pagelinks_only_where_candidates = pagelinks_only_where_candidates[pd.notna(pagelinks_only_where_candidates['wikidata_id'])]


parent_ids = pagelinks_only_where_candidates['parent_id'].drop_duplicates().values.tolist()

parent_ids_df = pd.DataFrame(parent_ids, columns=['parent_id'])


pagelinks_only_where_candidate_parents = pd.merge(en_pagelinks, parent_ids_df, how='left', left_on=['parent_id'], right_on=['parent_id'])

pagelinks_parents = pagelinks_only_where_candidate_parents[['parent_id', 'child_id']]

pagelinks_parents = pagelinks_parents.drop_duplicates()


# # convert en filtered pagelinks df to dictionary and save it
# pagelinks_only_parents_dict = {k: v['child_id'].tolist() for k,v in pagelinks_parents.groupby('parent_id')}

# # save en pagelinks dictionary as a pickle file
# with open(PATH_TO_DATA_EN+'pagelinks_only_parents_dict.p', 'wb') as fp:
#     pickle.dump(pagelinks_only_parents_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'en_pagelinks' is not defined

In [16]:
# load pickle file
with open(PATH_TO_DATA_EN+'pagelinks_only_parents_dict.p', 'rb') as fp:
    pagelinks_only_parents_dict = pickle.load(fp)

In [15]:
def get_concurrent_links_set(page_id):
    concurrent_set = set()
    
    for values in pagelinks_only_parents_dict.values():
        if page_id in values:
            concurrent_set = concurrent_set|set(values)

    return (page_id, concurrent_set)

In [28]:
# RUN ONLY ONCE!! Retrieves the concurrent links for candidates and saves it to files

num_cand_list = functions.divide_number(len(candidates_id_list), 100)

i = len(num_cand_list)
start_index = 0
num_items_proceed = num_cand_list[i-1]

while i > 0:
    start_time = time.time()
    print(i)
    print('start_index =', start_index)
    print('num_items_proceed =', num_items_proceed)
    
    candidates_list_part = candidates_id_list[start_index:num_items_proceed]

    p = Pool()
    candidate_concurrent_set = p.map(get_concurrent_links_set, candidates_list_part)
    p.close()
    p.join()

    # save concurrent sets as a pickle file
    with open(PATH_TO_DATA_EN+'concurrent_for_candidates/'+'candidate_concurrent_set_'+str(i)+'.p', 'wb') as fp:
         pickle.dump(candidate_concurrent_set, fp, protocol=pickle.HIGHEST_PROTOCOL)
            
    i -= 1
    start_index = num_items_proceed
    num_items_proceed = num_items_proceed + num_cand_list[i-1]
    print('Time for a set: %.1f minutes' % ((time.time() - start_time)/60))

In [74]:
# load pickle file
with open(PATH_TO_DATA_EN+'concurrent_for_candidates/'+'candidate_concurrent_set_100.p', 'rb') as fp:
    candidate_concurrent_set_test = pickle.load(fp)